In [18]:
import pandas as pd
import csv
from itertools import combinations
import re


# Read the csv file and functional dependencies file
file = pd.read_csv('exampleInputTable.csv')
print(file)
print('\n')

with open('data.txt', 'r') as f:
    lines = [line.strip() for line in f]

dependencies = {}
for line in lines:
    determ, depend = line.split(" -> ")
    # Seperating the determ by comma inorder to make a list
    determ = determ.split(", ")
    dependencies[tuple(determ)] = depend.split(", ")
print('Dependencies')
print(dependencies)
print('\n')


# Taking input from user
max_normal_form = input("Select the highest normal form to normalize (1: 1NF, 2: 2NF, 3: 3NF, B: BCNF, 4: 4NF, 5: 5NF): ")
if max_normal_form in ["1", "2", "3", "4", "5"]:
    max_normal_form = int(max_normal_form)

# To determine the highest normal form of the relation
find_high_normalform = int(
    input('Determine the highest normal form of the given relation? (1: Yes, 2: No): '))
high_normalform = 'not normalized'


# Enter Key
primary_key = input(
    "Enter the primary key values seperated with commas: ").split(', ')
print('\n')

keys = ()
for key in primary_key:
    keys = keys + (key,)

primary_key = keys


def contains_comma(series):
    return series.str.contains(',').any()


def parser(file):
    file = file.astype(str)
    columns_with_commas = [
        col for col in file.columns if contains_comma(file[col])]

    for col in columns_with_commas:
        file[col] = file[col].str.split(
            ',').apply(lambda x: [item.strip() for item in x])

    return file
file = parser(file)

multivalue = {}
if not max_normal_form == 'B' and max_normal_form >= 4:
    with open('mvd.txt', 'r') as fil:
        mvd_lines = [line.strip() for line in fil]

    print(mvd_lines)

    for mvd in mvd_lines:
        determ, depend = mvd.split(" ->> ")
        determ = determ.split(
            ", ") if ", " in determ else [determ]
        determ_str = str(determ)
        if determ_str in multivalue:
            multivalue[determ_str].append(depend)
        else:
            multivalue[determ_str] = [depend]


def is_list_or_set(item):
    return isinstance(item, (list, set))


def is_superkey(rel, determ):
    grouped = rel.groupby(
        list(determ)).size().reset_index(name='count')
    return not any(grouped['count'] > 1)


def powerset(s):
    x = len(s)
    for i in range(1 << x):
        yield [s[j] for j in range(x) if (i & (1 << j)) > 0]


def bcnf_decomp(rel, dependencies):
    for determ, depends in dependencies.items():
        if set(determ).issubset(rel.columns) and not is_superkey(rel, determ):
            depend_cols = list(determ) + depends
            new_rel1 = rel[depend_cols].drop_duplicates()
            remaining_cols = list(set(rel.columns) - set(depends))
            new_rel2 = rel[remaining_cols].drop_duplicates()
            return [new_rel1, new_rel2]
    return [rel]


def in_1nf(rel):
    if rel.empty:
        return False

    for column in rel.columns:
        unique_types = rel[column].apply(type).nunique()
        if unique_types > 1:
            return False
        if rel[column].apply(lambda x: isinstance(x, (list, dict, set))).any():
            return False

    return True


def in_2nf(primary_key, dependencies):
    for determs, depend in dependencies.items(): 
        # Check if X is a superkey
        superkey = True
        for x in determs:
            if x not in file.columns:
                superkey = False
                break
        if superkey:
            # Check if Y is fully depend on X
            for y in depend:
                if y not in file.columns or y not in determs:
                    return False
    return True 



def in_3nf(rels, dependencies):
    i = 0
    keys_as_list = list(dependencies.keys())
    for rel in rels:
        attributes = set(rels[rel].columns)
        non_prime_attributes = attributes - set(rel)
        i += 1

        for determ, depends in dependencies.items():
            if all(attr in non_prime_attributes for attr in determ):
                for depend in depends:
                    if depend in non_prime_attributes:
                        return False
    return True


def in_bcnf(rels, primary_key, dependencies):
    for rel in rels:
        for determ, depends in dependencies.items():
            if set(determ).issubset(rel.columns):
                if not is_superkey(rel, determ):
                    return False
    return True


def in_4nf(rels, multivalue):
    for rel in rels:
        for determ, depends in multivalue.items():
            for depend in depends:
                if isinstance(determ, tuple):
                    determ_cols = list(determ)
                else:
                    determ_cols = [determ]

                if all(col in rel.columns for col in determ_cols + [depend]):
                    grouped = rel.groupby(determ_cols)[
                        depend].apply(set).reset_index()
                    if len(grouped) < len(rel):
                        print(
                            f"violation of multi-valued dependencies: {determ} ->> {depend}")
                        return False

    return True


def in_5nf(rels):
    i = 0
    candidate_keys_dict = {}
    for rel in rels:
        print(rel)
        user_input = input("Enter the candidate keys ")
        print('\n')
        tuples = re.findall(r'\((.*?)\)', user_input)
        candidate_keys = [tuple(map(str.strip, t.split(','))) for t in tuples]
        candidate_keys_dict[i] = candidate_keys
        i += 1

    print(f'the Candidate Keys for given relation:')
    print(candidate_keys_dict)
    print('\n')

    j = 0
    for rel in rels:
        candidate_keys = candidate_keys_dict[j]
        j += 1

        data_tuples = [tuple(row) for row in rel.to_numpy()]

        
        def project(data, attributes):
            return {tuple(row[attr] for attr in attributes) for row in data}

        # to check if the given attributes are a super key
        def is_superkey(attributes):
            for key in candidate_keys:
                if set(key).issubset(attributes):
                    return True
            return False, candidate_keys_dict

        for i in range(1, len(rel.columns)):
            for attrs in combinations(rel.columns, i):
                
                if is_superkey(attrs):
                    continue

                # Project the data onto the attributes and their complement
                projected_data = project(data_tuples, attrs)
                complement_attrs = set(rel.columns) - set(attrs)
                complement_data = project(data_tuples, complement_attrs)
                # combine the projected data and check whether it is equals to the original data

                
                joined_data = {(row1 + row2)
                               for row1 in projected_data for row2 in complement_data}
                if set(data_tuples) != joined_data:
                    print("Not satisfy 5NF then check attribures:", attrs)
                    return False, candidate_keys_dict

    return True, candidate_keys_dict

def first_normal_form(rel):
    one_flag = in_1nf(rel)

    if one_flag:
        return rel, one_flag
    else:
        for col in rel.columns:
            if rel[col].apply(is_list_or_set).any():
                rel = rel.explode(col)

        return rel, one_flag

def second_normal_form(rel, primary_key, dependencies):
    rels = {}
    original_rel = rel
    two_flag = in_2nf(primary_key, dependencies)

    if two_flag:
        rels[primary_key] = rel
        return rels, two_flag
    else:
        depend_keys = list(dependencies.keys())
        for determ, depends in dependencies.items():
            modified_depends = [
                dep + '_fk' if (dep,) in depend_keys else dep for dep in depends]

            cols = list(determ) + depends
            rels[tuple(determ)] = rel[cols].drop_duplicates(
            ).reset_index(drop=True)

            rename_dict = {dep: modified_dep for dep,
                           modified_dep in zip(depends, modified_depends)}
            rels[tuple(determ)].rename(
                columns=rename_dict, inplace=True)

        junc_cols = []
        rel_name = ''
        for rel in rels:
            if set(rel).issubset(primary_key):
                rel_name += "_".join(rel)
                junc_cols.append(rel)

        if len(junc_cols) > 1:
            jun_cols = list(junc_cols)
            cols = [element for tup in jun_cols for element in tup]
            temp_df = original_rel[cols].drop_duplicates(
            ).reset_index(drop=True)

            renamed_cols = [col + '_fk' for col in cols]
            temp_df.columns = renamed_cols + \
                [col for col in temp_df.columns if col not in cols]

            temp_df[rel_name] = range(1, len(temp_df) + 1)
            col_order = [rel_name] + renamed_cols
            temp_df = temp_df[columns_order]
            rels[rel_name] = temp_df


        return rels, two_flag


def third_normal_form(rels, primary_key, dependencies):
    three_rels = {}
    original_rel = rels
    three_flag = in_3nf(rels, dependencies)

    if three_flag:
        return rels, three_flag
    else:
        depend_keys = list(dependencies.keys())
        for rel in rels:
            original_rel = rels[rel]
            for determ, depends in dependencies.items():
                modified_depends = [
                    dep + '_fk' if (dep,) in depend_keys else dep for dep in depends]

                cols = list(determ) + depends
                three_rels[tuple(determ)] = rels[rel][cols].drop_duplicates(
                ).reset_index(drop=True)

                rename_dict = {dep: modified_dep for dep,
                               modified_dep in zip(depends, modified_depends)}
                three_rels[tuple(determ)].rename(
                    columns=rename_dict, inplace=True)

        junc_cols = []

        rel_name = ''
        for rel in three_rel:
            rel_name += "_".join(rel)
            junc_cols.append(rel)

        if len(junc_cols) > 1:
            jun_cols = list(junc_cols)
            cols = [element for tup in jun_cols for element in tup]
            temp_df = original_rel[cols].drop_duplicates(
            ).reset_index(drop=True)

            renamed_cols = [col + '_fk' for col in cols]
            temp_df.columns = renamed_cols + \
                [col for col in temp_df.columns if col not in cols]

            temp_df[rel_name] = range(1, len(temp_df) + 1)
            col_order = [rel_name] + renamed_cols
            temp_df = temp_df[columns_order]
            three_rels[rel_name] = temp_df

        return three_rels, three_flag


def bc_normal_form(rels, primary_key, dependencies):
    rels = list(rels.values())
    bcnf_rels = []
    bcnf_flag = in_bcnf(rels, primary_key, dependencies)

    if bcnf_flag:
        return rels, bcnf_flag
    else:
        for rel in rels:
            bcnf_decomp_rel = bcnf_decomp(
                rel, dependencies)
            if len(bcnf_decomp_rel) == 1:
                bcnf_rels.append(bcnf_decomp_rel)
            else:
                rels.extend(bcnf_decomp_rel)

    return bcnf_rels, bcnf_flag


def fourth_normal_form(rels, multivalue):
    four_rels = []
    four_flag = in_4nf(rels, multivalue)

    if four_flag:
        return rels, four_flag
    else:
        for rel in rels:
            for determ, depends in multivalue.items():
                for depend in depends:
                    if isinstance(determ, tuple):
                        determ_cols = list(determ)
                    else:
                        determ_cols = [determ]

                    if all(col in rel.columns for col in determ_cols + [depend]):
                        grouped = rel.groupby(determ_cols)[
                            depend].apply(set).reset_index()
                        if len(grouped) < len(rel):
                            # Decomposition
                            table_1 = rel[determ_cols +
                                               [depend]].drop_duplicates()
                            table_2 = rel[determ_cols + [col for col in rel.columns if col not in [
                                depend] + determ_cols]].drop_duplicates()

                        
                            four_rels.extend([table_1, table_2])

                            break
                else:
                    continue
                break
            else:
                four_rels.append(rel)

    if len(four_rels) == len(rels):
        return four_rels  # relations are in 4NF
    else:
        return fourth_normal_form(four_rels, multivalue)


def decomposing_to_5nf(dataframe, candidate_keys):
    
    def project(df, attributes):
        return df[list(attributes)].drop_duplicates().reset_index(drop=True)

    # find whether the decomposition is loseless
    def is_lossless(df, df1, df2):
        common_columns = set(df1.columns) & set(df2.columns)
        if not common_columns:
            return False
        joined_df = pd.merge(df1, df2, how='inner', on=list(common_columns))
        return df.equals(joined_df)

    decomposed_tables = [dataframe]

    # check for each candidate key and thn decompose the table
    for key in candidate_keys:
        new_tables = []
        for table in decomposed_tables:
            if set(key).issubset(set(table.columns)):
                table1 = project(table, key)
                remaining_columns = set(table.columns) - set(key)
                table2 = project(table, remaining_columns | set(key))

                # Check if the decomposition is lossless
                if is_lossless(table, table1, table2):
                    new_tables.extend([table1, table2])
                else:
                    new_tables.append(table)
            else:
                new_tables.append(table)
        decomposed_tables = new_tables

    return decomposed_tables


def fifth_normal_form(rels, primary_key, dependencies):
    five_rels = []
    five_flag, candidate_keys_dict = in_5nf(rels)

    if five_flag:
        return rels, five_flag
    else:
        i = 0
        for rel in rels:
            candidate_keys = candidate_keys_dict[i]
            i += 1
            decomposed_rels = decomposing_to_5nf(rel, candidate_keys)
            five_rels.append(decomposed_rels)

    return five_rels, five_flag

# Used to generate output as per the requirement
def output(dtype):
    """change pandas dtype to SQL data type."""
    if "int" in str(dtype):
        return "INT"
    elif "float" in str(dtype):
        return "FLOAT"
    elif "object" in str(dtype):
        return "VARCHAR(255)"
    elif "datetime" in str(dtype):
        return "DATETIME"
    else:
        return "TEXT"


def sql_query_1NF(primary_keys, df):
    t_name = "_".join(primary_keys) + "_table"

    # Create SQL Query
    query = f"CREATE TABLE {t_name} (\n"

    # Iterate through columns to create query
    for col, dtype in zip(df.columns, df.dtypes):
        if col in primary_keys:
            query += f"  {col} {output(dtype)} PRIMARY KEY,\n"
        else:
            query += f"  {col} {output(dtype)},\n"

    
    query = query.rstrip(',\n') + "\n);"

    print(query)



def sql_query_2_3(rels):
    for rel in rels:
        primary_keys = rel
        primary_keys = (primary_keys,) if isinstance(
        primary_keys, str) else primary_keys
        t_name = "_".join(primary_keys) + '_table'
        rel = rels[rel]

        # Create SQL Query
        query = f"CREATE TABLE {t_name} (\n"

        # Iterate through columns to create query
        for col, dtype in zip(rel.columns, rel.dtypes):
            if col in primary_keys:
                query += f"  {col} {output(dtype)} PRIMARY KEY,\n"
            elif '_fk' in col:
                query += f" FOREIGN KEY ({col}) REFERENCES {col.replace('_fk','')}_table({col.replace('_fk','')}),\n"
            else:
                query += f"  {col} {output(dtype)},\n"

        
        query = query.rstrip(',\n') + "\n);"

        print(query)


def sql_query_BCNF_4_5(rels):
    for rel in rels:
        primary_key = rel.columns[0]
        t_name = f'{primary_key}_table'

        # Create SQL Query
        query = f"CREATE TABLE {t_name} (\n"

        # Iterate through columns to create query
        for col, dtype in zip(rel.columns, rel.dtypes):
            if col == primary_key:
                query += f"  {col} {output(dtype)} PRIMARY KEY,\n"
            elif '_fk' in col:
                query += f" FOREIGN KEY ({col}),\n"
            else:
                query += f"  {col} {output(dtype)},\n"

        
        query = query.rstrip(',\n') + "\n);"

        print(query)

if max_normal_form == 'B' or max_normal_form >= 1:
    onenf_table, one_flag = first_normal_form(
        file)
    if one_flag:
        high_normalform = 'The Highest Normal Form of the given table is: 1NF'
        
    if max_normal_form == 1:
        if one_flag:
            print('The table is already in 1NF')
            print('\n')

        sql_query_1NF(primary_key, onenf_table)

if max_normal_form == 'B' or max_normal_form >= 2:
    twonf_tables, two_flag = second_normal_form(
        onenf_table, primary_key, dependencies)
    if one_flag and two_flag:
        high_normalform = 'The Highest Normal Form of the given table is: 2NF'

    if max_normal_form == 2:
        if two_flag and one_flag:
            print('The table is already in 2NF')
            print('\n')

        sql_query_2_3(twonf_tables)

if max_normal_form == 'B' or max_normal_form >= 3:
    threenf_tables, three_flag = third_normal_form(
        twonf_tables, primary_key, dependencies)
    
    if one_flag and two_flag and three_flag:
        high_normalform = 'The Highest Normal Form of the given table is: 3NF'
        
    if max_normal_form == 3:
        if three_flag and two_flag and one_flag:
            print('The table is already in 3NF')
            print('\n')

        sql_query_2_3(threenf_tables)

if max_normal_form == 'B' or max_normal_form >= 4:
    bcnf_tables, bcnf_flag = bc_normal_form(
        threenf_tables, primary_key, dependencies)
    
    if one_flag and two_flag and three_flag and bcnf_flag:
        high_normalform = 'The Highest Normal Form of the given table is: BCNF'
        
    if max_normal_form == 'B':
        if bcnf_flag and three_flag and two_flag and one_flag:
            print('The table is already in BCNF')
            print('\n')

        sql_query_BCNF_4_5(bcnf_tables)

if not max_normal_form == 'B' and max_normal_form >= 4:
    fournf_tables, four_flag = fourth_normal_form(
        bcnf_tables, multivalue)
    
    if one_flag and two_flag and three_flag and bcnf_flag and four_flag:
        high_normalform = 'The Highest Normal Form of the given table is: 4NF'
        
    if max_normal_form == 4:
        if four_flag and bcnf_flag and three_flag and two_flag and one_flag:
            print('The table is already in 4NF')
            print('\n')

        sql_query_BCNF_4_5(fournf_tables)

if not max_normal_form == 'B' and max_normal_form >= 5:
    fivenf_tables, five_flag = fifth_normal_form(
        fournf_tables, primary_key, dependencies)
    
    if one_flag and two_flag and three_flag and bcnf_flag and four_flag and five_flag:
        high_normalform = 'The Highest Normal Form of the given table is: 5NF'
        
    if max_normal_form == 5:
        if five_flag and four_flag and bcnf_flag and three_flag and two_flag and one_flag:
            print('The table is already in 5NF')
            print('\n')

        sql_query_BCNF_4_5(fivenf_tables)
        
if find_high_normalform == 1:
    print('\n')
    print(high_normalform)
    print('\n')


   StudentID FirstName  LastName   Course  Professor  ProfessorEmail  \
0        101      John       Doe  Math101   Dr.Smith   smith@mst.edu   
1        102      Jane       Roe  Math101   Dr.Smith   smith@mst.edu   
2        103   Arindam    Khanda    CS101   Dr.Jones   jones@mst.edu   
3        104      Jose  Franklin   Bio101  Dr.Watson  watson@mst.edu   
4        105       Ada  Lovelace    CS101   Dr.Jones   jones@mst.edu   

  CourseStart CourseEnd  
0      1/1/23   5/30/23  
1      1/1/23   5/30/23  
2      2/1/23   6/15/23  
3      3/1/23   7/20/23  
4      2/1/23   6/15/23  


Dependencies
{('StudentID',): ['FirstName', 'LastName'], ('Course',): ['CourseStart', 'CourseEnd', 'Professor'], ('Professor',): ['ProfessorEmail']}




Select the highest normal form to normalize (1: 1NF, 2: 2NF, 3: 3NF, B: BCNF, 4: 4NF, 5: 5NF):  B
Determine the highest normal form of the given relation? (1: Yes, 2: No):  1
Enter the primary key values seperated with commas:  StudentID,Course




CREATE TABLE StudentID_table (
  StudentID VARCHAR(255) PRIMARY KEY,
  FirstName VARCHAR(255),
  LastName VARCHAR(255)
);
CREATE TABLE Course_table (
  Course VARCHAR(255) PRIMARY KEY,
  CourseStart VARCHAR(255),
  CourseEnd VARCHAR(255),
 FOREIGN KEY (Professor_fk)
);
CREATE TABLE Professor_table (
  Professor VARCHAR(255) PRIMARY KEY,
  ProfessorEmail VARCHAR(255)
);


The Highest Normal Form of the given table is: 1NF


